## NBA GM Draft Picks Analysis - Part 2: 2022-2024 Draft Picks

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import requests
import time
import json
from bs4 import BeautifulSoup

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
years = list(range(2022,2025))

In [ ]:
winshares_df = pd.read_csv('../inputs/win_shares_stats_2022_2024.csv')

In [ ]:
draft_df = pd.read_csv('../inputs/gm_draftpicks_2022_2024.csv')

### Merging Data

In [15]:
merged_df = pd.merge(draft_df, winshares_df[['Player', 'Year', 'Tm', 'WS']], 
                     left_on=['Acquired Player', 'Season', 'Team Acquiring'], 
                     right_on=['Player', 'Year', 'Tm'], 
                     how='left')

# Assign the WS values to Acquired Player BPM/WARP Year 1 column
draft_df['Acquired Player Win Shares Year 1'] = merged_df['WS']

In [16]:
merged_df_year2 = pd.merge(draft_df, winshares_df[['Player', 'Year', 'Tm', 'WS']], 
                     left_on=['Acquired Player', 'Year 2', 'Team Acquiring'], 
                     right_on=['Player', 'Year', 'Tm'], 
                     how='left')

draft_df['Acquired Player Win Shares Year 2'] = merged_df_year2['WS']

In [17]:
merged_df_year3 = pd.merge(draft_df, winshares_df[['Player', 'Year', 'Tm', 'WS']], 
                     left_on=['Acquired Player', 'Year 3', 'Team Acquiring'], 
                     right_on=['Player', 'Year', 'Tm'], 
                     how='left')

draft_df['Acquired Player Win Shares Year 3'] = merged_df_year3['WS']

In [20]:
draft_df[draft_df['Acquired Player'] == 'Cade Cunningham']

,Season,Team Acquiring,Acquired Player,Draft Pick,Acquisition Type,Acquiring GM,Year 2,Year 3,Helper,Acquired Player Win Shares Year 1,Acquired Player Win Shares Year 2,Acquired Player Win Shares Year 3
0,2022,DET,Cade Cunningham,1,Draft Pick,Troy Weaver,2023,2024,2022Cade Cunningham,-0.5,-0.1,1.8


In [21]:
winshares_df[winshares_df['Player'] == 'Cade Cunningham']

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year
161,Cade Cunningham,SG,20,DET,64,2088,13.1,.504,.352,.163,2.9,16.2,9.2,29.1,1.8,1.9,17.5,27.5,-2.4,1.9,-0.5,-0.011,-1.1,-0.5,-1.6,0.2,2022
984,Cade Cunningham,PG,21,DET,12,400,14.0,.492,.272,.192,3.2,17.7,10.3,30.7,1.2,1.5,13.8,30.0,-0.4,0.2,-0.1,-0.018,-0.1,-0.6,-0.7,0.1,2023
1701,Cade Cunningham,PG,22,DET,62,2074,17.1,.546,.287,.235,1.7,12.9,7.2,37.5,1.3,0.9,14.1,30.8,0.7,1.0,1.8,.041,2.0,-1.6,0.3,1.2,2024


In [22]:
draft_df.isna().sum()

Season                                 0
Team Acquiring                         0
Acquired Player                        0
Draft Pick                             0
Acquisition Type                       0
Acquiring GM                           0
Year 2                                 0
Year 3                                 0
Helper                                 0
Acquired Player Win Shares Year 1     28
Acquired Player Win Shares Year 2     88
Acquired Player Win Shares Year 3    145
dtype: int64

In [23]:
draft_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Season                             176 non-null    int64 
 1   Team Acquiring                     176 non-null    object
 2   Acquired Player                    176 non-null    object
 3   Draft Pick                         176 non-null    int64 
 4   Acquisition Type                   176 non-null    object
 5   Acquiring GM                       176 non-null    object
 6   Year 2                             176 non-null    int64 
 7   Year 3                             176 non-null    int64 
 8   Helper                             176 non-null    object
 9   Acquired Player Win Shares Year 1  148 non-null    object
 10  Acquired Player Win Shares Year 2  88 non-null     object
 11  Acquired Player Win Shares Year 3  31 non-null     object
dtypes: int64

From here it looks about half of players (88/176) in this dataset did not produce win shares on the same team in year 2 after the acquisition.

In [24]:
draft_df[draft_df['Acquired Player Win Shares Year 2'].notna()]

,Season,Team Acquiring,Acquired Player,Draft Pick,Acquisition Type,Acquiring GM,Year 2,Year 3,Helper,Acquired Player Win Shares Year 1,Acquired Player Win Shares Year 2,Acquired Player Win Shares Year 3
0,2022,DET,Cade Cunningham,1,Draft Pick,Troy Weaver,2023,2024,2022Cade Cunningham,-0.5,-0.1,1.8
1,2022,HOU,Jalen Green,2,Draft Pick,Rafael Stone,2023,2024,2022Jalen Green,0.7,1.8,3.1
2,2022,CLE,Evan Mobley,3,Draft Pick,Koby Altman,2023,2024,2022Evan Mobley,5.2,8.5,5.5
3,2022,TOR,Scottie Barnes,4,Draft Pick,Bobby Webster,2023,2024,2022Scottie Barnes,6.6,5.0,4.3
4,2022,ORL,Jalen Suggs,5,Draft Pick,John Hammond,2023,2024,2022Jalen Suggs,-1.6,1.4,4.7
...,...,...,...,...,...,...,...,...,...,...,...,...
107,2023,IND,Kendall Brown,48,Draft Pick,Chad Buchanan,2024,2025,2023Kendall Brown,0.1,0.1,NaN
108,2023,CLE,Isaiah Mobley,49,Draft Pick,Koby Altman,2024,2025,2023Isaiah Mobley,0.2,0.0,NaN
112,2023,BOS,JD Davison,53,Draft Pick,Brad Stevens,2024,2025,2023JD Davison,0.1,0.2,NaN
114,2023,GSW,Gui Santos,55,Draft Pick,Mike Dunleavy Jr.,2024,2025,2023Gui Santos,NaN,0.7,NaN


In [25]:
draft_df[draft_df['Acquired Player Win Shares Year 1'].isna()].sort_values(['Season', 'Draft Pick'], ascending=True)

,Season,Team Acquiring,Acquired Player,Draft Pick,Acquisition Type,Acquiring GM,Year 2,Year 3,Helper,Acquired Player Win Shares Year 1,Acquired Player Win Shares Year 2,Acquired Player Win Shares Year 3
32,2022,LAC,Jason Preston,33,Draft Pick,Michael Winger,2023,2024,2022Jason Preston,NaN,0.1,NaN
33,2022,NOP,Rokas Jokubaitis,34,Draft Pick,Trajan Langdon,2023,2024,2022Rokas Jokubaitis,NaN,NaN,NaN
44,2022,BOS,Juhann Begarin,45,Draft Pick,Brad Stevens,2023,2024,2022Juhann Begarin,NaN,NaN,NaN
48,2022,BRK,Marcus Zegarowski,49,Draft Pick,Sean Marks,2023,2024,2022Marcus Zegarowski,NaN,NaN,NaN
49,2022,PHI,Filip Petrušev,50,Draft Pick,Daryl Morey,2023,2024,2022Filip Petrušev,NaN,NaN,NaN
56,2022,DET,Balša Koprivica,57,Draft Pick,Troy Weaver,2023,2024,2022Balša Koprivica,NaN,NaN,NaN
58,2022,BRK,RaiQuan Gray,59,Draft Pick,Sean Marks,2023,2024,2022RaiQuan Gray,NaN,0.1,NaN
61,2023,OKC,Chet Holmgren,2,Draft Pick,Sam Presti,2024,2025,2023Chet Holmgren,NaN,8.9,NaN
86,2023,MIA,Nikola Jovic,27,Draft Pick,Andy Elisburg,2024,2025,2023Nikola Jovic,NaN,NaN,NaN
95,2023,POR,Gabriele Procida,36,Draft Pick,Joe Cronin,2024,2025,2023Gabriele Procida,NaN,NaN,NaN


In [26]:
draft_df.loc[draft_df["Acquired Player"] == "Nikola Jovic", "Acquired Player Win Shares Year 1"] = 0.2
draft_df.loc[draft_df["Acquired Player"] == "Nikola Jovic", "Acquired Player Win Shares Year 2"] = 1.9

In [28]:
draft_df['Acquired Player Win Shares Year 1'] = draft_df['Acquired Player Win Shares Year 1'].fillna(0)
draft_df['Acquired Player Win Shares Year 2'] = draft_df['Acquired Player Win Shares Year 2'].fillna(0)
draft_df['Acquired Player Win Shares Year 3'] = draft_df['Acquired Player Win Shares Year 3'].fillna(0)

I filled in win share data for the only relevant rookie missing then set the null values to 0.

In [ ]:
expected_winshares = pd.read_csv('../outputs/draft_pick_expected_win_shares.csv')

In [30]:
expected_winshares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Pk             61 non-null     int64  
 1   Rookie_WS      61 non-null     float64
 2   Sophomore_WS   61 non-null     float64
 3   Third_WS       61 non-null     float64
 4   Sophomore_AVG  61 non-null     float64
 5   Third_AVG      61 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 3.0 KB


In [32]:
draft_df = pd.merge(draft_df, expected_winshares, 
                     left_on='Draft Pick', 
                     right_on='Pk', 
                     how='left')

In [33]:
draft_df.columns

Index(['Season', 'Team Acquiring', 'Acquired Player', 'Draft Pick',
       'Acquisition Type', 'Acquiring GM', 'Year 2', 'Year 3', 'Helper',
       'Acquired Player Win Shares Year 1',
       'Acquired Player Win Shares Year 2',
       'Acquired Player Win Shares Year 3', 'Pk', 'Rookie_WS', 'Sophomore_WS',
       'Third_WS', 'Sophomore_AVG', 'Third_AVG'],
      dtype='object')

In [38]:
draft_df['Year 2_AVG'] = (draft_df['Acquired Player Win Shares Year 1'] + draft_df['Acquired Player Win Shares Year 2']) / 2
draft_df['Year 3_AVG'] = (draft_df['Acquired Player Win Shares Year 1'] + draft_df['Acquired Player Win Shares Year 2'] + draft_df['Acquired Player Win Shares Year 3']) / 3

I created year 2 and year 3 averages, which is an average of win shares over the first two years of a player's career and an average of win shares over the first three years of a player's career. This will allow me to have one metric to compare to as opposed to multiple columns later on. The rookies will be judged on year 1 win shares, the sophomores will be judged on year 2 average, and the third years will be judged on year 3 average.

In [ ]:
draft_df.to_csv('../outputs/draft_pick_win_shares_2022_2024.csv', index=False)

In [ ]:
draft_df = pd.read_csv('../outputs/draft_pick_win_shares_2022_2024.csv')

In [41]:
draft_df

,Season,Team Acquiring,Acquired Player,Draft Pick,Acquisition Type,Acquiring GM,Year 2,Year 3,Helper,Acquired Player Win Shares Year 1,Acquired Player Win Shares Year 2,Acquired Player Win Shares Year 3,Pk,Rookie_WS,Sophomore_WS,Third_WS,Sophomore_AVG,Third_AVG,Year 2_AVG,Year 3_AVG
0,2022,DET,Cade Cunningham,1,Draft Pick,Troy Weaver,2023,2024,2022Cade Cunningham,-0.5,-0.1,1.8,1,1.454655,2.775450,3.215094,2.115052,2.481733,-0.30,0.400000
1,2022,HOU,Jalen Green,2,Draft Pick,Rafael Stone,2023,2024,2022Jalen Green,0.7,1.8,3.1,2,1.429293,2.727405,3.161436,2.078349,2.439378,1.25,1.866667
2,2022,CLE,Evan Mobley,3,Draft Pick,Koby Altman,2023,2024,2022Evan Mobley,5.2,8.5,5.5,3,1.403932,2.679359,3.107778,2.041646,2.397023,6.85,6.400000
3,2022,TOR,Scottie Barnes,4,Draft Pick,Bobby Webster,2023,2024,2022Scottie Barnes,6.6,5.0,4.3,4,1.378571,2.631314,3.054121,2.004942,2.354668,5.80,5.300000
4,2022,ORL,Jalen Suggs,5,Draft Pick,John Hammond,2023,2024,2022Jalen Suggs,-1.6,1.4,4.7,5,1.353210,2.583268,3.000463,1.968239,2.312314,-0.10,1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,2024,SAC,Jalen Slawson,54,Draft Pick,Monte McNair,2025,2026,2024Jalen Slawson,0.0,0.0,0.0,54,0.110512,0.229040,0.371228,0.169776,0.236926,0.00,0.000000
172,2024,IND,Isaiah Wong,55,Draft Pick,Chad Buchanan,2025,2026,2024Isaiah Wong,0.0,0.0,0.0,55,0.085151,0.180994,0.317570,0.133072,0.194572,0.00,0.000000
173,2024,MEM,Tarik Biberovic,56,Draft Pick,Zach Kleiman,2025,2026,2024Tarik Biberovic,0.0,0.0,0.0,56,0.059790,0.132949,0.263912,0.096369,0.152217,0.00,0.000000
174,2024,GSW,Trayce Jackson-Davis,57,Draft Pick,Mike Dunleavy Jr.,2025,2026,2024Trayce Jackson-Davis,4.8,0.0,0.0,57,0.034428,0.084903,0.210254,0.059666,0.109862,2.40,1.600000
